# Business Cycle

## Imports

In [1]:
path = "../Data_Ryan"
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pmp_functions_v4 as pmp

## Data Cleaning

In [2]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Equity Returns.xlsx",
    sheet_name = "WORLD - MXWO Index",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)

display(benchmark_data)

,Price (USD)
Date,
1970-01-30,94.2500
1970-02-27,96.9800
1970-03-31,97.0700
1970-04-30,87.8000
1970-05-29,82.0600
...,...
2025-07-31,8057.2246
2025-08-29,8269.7393
2025-09-30,8538.7900


In [3]:
# --- Load Factors Data ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100
display(factors_data)

# --- Riskfree Rate ---
riskfree = factors_data["RF"]
riskfree = riskfree.to_frame()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_65430/2744343010.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


,Mkt-RF,SMB,HML,RF
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031
...,...,...,...,...
2025-06-30,0.0486,0.0083,-0.0160,0.0034
2025-07-31,0.0198,0.0027,-0.0127,0.0034
2025-08-31,0.0184,0.0387,0.0442,0.0038
2025-09-30,0.0339,-0.0184,-0.0105,0.0033


,RF
1926-07-31,0.0022
1926-08-31,0.0025
1926-09-30,0.0023
1926-10-31,0.0032
1926-11-30,0.0031
...,...
2025-06-30,0.0034
2025-07-31,0.0034
2025-08-31,0.0038
2025-09-30,0.0033


In [4]:
# --- Load Macro Data ---
CPI_realized = pd.read_excel(
    f"{path}/Business Cycle Indicator.xlsx",
    sheet_name = 'CPI Realized',
    index_col = 0,
    parse_dates = True
)
CPI_realized.index = pd.to_datetime(CPI_realized.index)
CPI_realized.index = CPI_realized.index + pd.offsets.MonthEnd(0)

RGDP_realized = pd.read_excel(
    f"{path}/Business Cycle Indicator.xlsx",
    sheet_name = 'RGDP Realized',
    index_col = 0,
    parse_dates = True
)
RGDP_realized.index = pd.to_datetime(RGDP_realized.index)
RGDP_realized.index = RGDP_realized.index + pd.offsets.MonthEnd(0)

display("CPI Realized:")
display(CPI_realized)
display("RGDP Realized:")
display(RGDP_realized)

'CPI Realized:'

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,6.16246,4.95739,2.31338,7.75127,NaN,NaN
1970-02-28,6.42458,4.93065,2.12360,7.75127,NaN,NaN
1970-03-31,6.09418,5.14198,2.49918,7.71710,NaN,NaN
1970-04-30,6.06061,5.61884,2.59281,7.68362,NaN,NaN
1970-05-31,6.04396,6.08365,3.13683,7.24558,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,2.67268,4.10000,0.10000,3.30000,2.1,1.96760
2025-07-31,2.73180,4.20000,0.20000,3.10000,3.2,2.00791
2025-08-31,2.93922,4.10000,0.20000,2.70000,3.2,2.02889


'RGDP Realized:'

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,0.32279,1.14290,NaN,NaN,8.25550,NaN
1970-04-30,0.16112,2.78091,NaN,NaN,8.29844,NaN
1970-07-31,0.42134,3.20575,NaN,NaN,6.31359,NaN
1970-10-31,-0.16735,3.65875,NaN,NaN,4.39906,NaN
1971-01-31,2.69716,3.80887,NaN,NaN,3.52489,NaN
...,...,...,...,...,...,...
2024-07-31,2.79139,1.15839,1.88497,0.39838,0.76075,0.96404
2024-10-31,2.39979,1.46538,1.66876,1.12419,1.25191,1.32560
2025-01-31,2.01927,1.29504,2.43952,1.83565,1.33678,1.62167


In [5]:
# --- Load Equity Prices ---
equity_map = {
    'US': 'US - SPX Index',
    'UK': 'UK - MXGB Index',
    'EU': 'EU - MXEM Index',
    'CH': 'CH - MXCH Index',
    'JP': 'JP - MXJP Index',
    'AU': 'AU - MXAU Index',
    'EM': 'EM - MXEF Index'
}

equity_price_list = []
for country, sheet in equity_map.items():
    df = pd.read_excel(
        f"{path}/Equity Returns.xlsx",
        sheet_name = sheet,
        index_col = 0,
        parse_dates = True
    )

    last_price = df.iloc[:, 0].resample('ME').last()
    last_price.name = country
    equity_price_list.append(last_price)

equity_prices = pd.concat(equity_price_list, axis=1)

display("Equity Prices:")
display(equity_prices)

'Equity Prices:'

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,85.0200,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,89.7600,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,90.1511,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,82.2655,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,77.5125,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,30137.0645,10600.6826,488.1158,3991.8601,19.8688,6202.4824,2695.0002
2025-08-31,30747.9980,10995.8330,502.1103,4170.1406,21.2656,6451.0068,2734.5061
2025-09-30,31867.3594,11134.9863,518.8801,4160.9258,21.8248,6435.8457,2930.9309


In [6]:
# --- Load Currency Prices ---
currency_map = {
    'AU': 'AUDUSD Curncy',
    'JP': 'JPYUSD Curncy',
    'CH': 'CHFUSD Curncy',
    'EU': 'EURUSD Curncy',
    'UK': 'GBPUSD Curncy'
}

currency_list = []
for country, sheet in currency_map.items():
    df = pd.read_excel(
        f"{path}/FX.xlsx",
        sheet_name = sheet,
        index_col = 0,
        parse_dates = True
    )

    columns = ['Spot', 'FW Points']
    df = df[columns].copy()

    new_cols = {
        'Spot': f'{country} Spot',
        'FW Points': f'{country} Fwd Pts'
    }
    df.rename(columns=new_cols, inplace=True)
    currency_list.append(df)

currency_list = pd.concat(currency_list, axis = 1).resample('ME').last()
display(currency_list)

,AU Spot,AU Fwd Pts,JP Spot,JP Fwd Pts,CH Spot,CH Fwd Pts,EU Spot,EU Fwd Pts,UK Spot,UK Fwd Pts
Date,,,,,,,,,,
1971-01-31,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4174,NaN
1971-02-28,NaN,NaN,NaN,NaN,0.2322,NaN,NaN,NaN,2.4160,NaN
1971-03-31,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4168,NaN
1971-04-30,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4193,NaN
1971-05-31,NaN,NaN,NaN,NaN,0.2436,NaN,NaN,NaN,2.4178,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.6581,4.14,0.006943,-53.06,1.2609,-32.86,1.1787,25.69,1.3732,2.00
2025-07-31,0.6425,3.68,0.006634,-51.87,1.2311,-31.57,1.1415,23.52,1.3207,4.00
2025-08-31,0.6540,3.43,0.006800,-48.78,1.2492,-29.60,1.1686,22.54,1.3504,3.87


## Global Variables

In [7]:
frequency = 1
t_cost = 0
k = 2
weights_lag = 0
window = 12*20
short = True
beta_neutral = False
min_regions = 4
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [8]:
# --- Compute Business Cycle Signal ---
CPI_trend = CPI_realized.diff(12)
CPI_trend_lagged = CPI_trend.shift(3)

RGDP_trend = RGDP_realized.diff(4)
# .resample('MS').ffill() to convert quarterly to monthly by forward filling
RGDP_trend_lagged = RGDP_trend.shift(1).resample('MS').ffill().resample('ME').last()

# Reindex RGDP trend to match CPI trend index exactly
RGDP_component = RGDP_trend_lagged.reindex(CPI_trend_lagged.index).ffill()
CPI_component = CPI_trend_lagged

In [9]:
# --- Equities Composite Signal Construction ---
# Logic: For Equities, we want High Growth (+) and Low Inflation (-)
# We combine them: 50% Growth, 50% Inverse Inflation
business_cyle_signal_equities = (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal_equities = business_cyle_signal_equities.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal_equities)

'Business Cycle Signal:'

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,0.109695,0.490235,1.529800,1.052315,0.699510,0.690380
2025-07-31,0.088140,-0.222910,1.845270,0.952315,0.947685,0.664725
2025-08-31,0.009460,-0.298515,1.923505,1.152315,0.947685,0.898805


In [10]:
# --- Currencies Composite Signal Construction ---
# Logic: For Currencies, we want High Growth (+) and High(ish) Inflation (-)
business_cyle_signal_currencies = (0.5 * RGDP_component) + (0.5 * CPI_component)
business_cyle_signal_currencies = business_cyle_signal_currencies.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal_currencies)

'Business Cycle Signal:'

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-0.953715,0.124195,0.827170,1.952315,-0.518590,0.434070
2025-07-31,-0.932160,0.837340,0.511700,2.052315,-0.766765,0.459725
2025-08-31,-0.853480,0.912945,0.433465,1.852315,-0.766765,0.225645


In [11]:
# --- Bonds Composite Signal Construction ---
# Logic: For Bonds, we want Low Growth (-) and Low Inflation (-)
business_cyle_signal_bonds = - (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal_bonds = business_cyle_signal_bonds.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal_bonds)

'Business Cycle Signal:'

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,0.953715,-0.124195,-0.827170,-1.952315,0.518590,-0.434070
2025-07-31,0.932160,-0.837340,-0.511700,-2.052315,0.766765,-0.459725
2025-08-31,0.853480,-0.912945,-0.433465,-1.852315,0.766765,-0.225645


In [12]:
# --- Interest Rates Composite Signal Construction ---
# Logic: For Interest, we want Low Growth (-) and Low Inflation (-)
business_cyle_signal_int = - (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal_int = business_cyle_signal_int.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal_int)

'Business Cycle Signal:'

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,0.953715,-0.124195,-0.827170,-1.952315,0.518590,-0.434070
2025-07-31,0.932160,-0.837340,-0.511700,-2.052315,0.766765,-0.459725
2025-08-31,0.853480,-0.912945,-0.433465,-1.852315,0.766765,-0.225645


## Asset Class Returns

In [13]:
# --- Equity Returns ---
equity_returns = equity_prices.pct_change()
equity_returns = equity_returns.resample('ME').last()

display(equity_returns)

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,0.055752,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,0.004357,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,-0.087471,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,-0.057776,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,0.022443,0.007016,-0.018617,-0.027445,-0.016742,-0.003080,0.020152
2025-08-31,0.020272,0.037276,0.028670,0.044661,0.070301,0.040069,0.014659
2025-09-30,0.036404,0.012655,0.033399,-0.002210,0.026296,-0.002350,0.071832


In [14]:
# --- Currency Returns ---
excess_returns_list = []
countries = ['AU', 'JP', 'CH', 'EU', 'UK'] 
divisor_map = {'JP': 100} # JPY uses 100, all others default to 10000

for country in countries:
    # 1. Get the relevant columns
    spot_col = f'{country} Spot'
    fwd_col = f'{country} Fwd Pts'

    # Determine the correct divisor (100 for JPY, 10000 for others)
    divisor = divisor_map.get(country, 10000)
    
    # 2. Calculate the Spot Return (EOM data, no shift needed)
    # Spot Return = (S_t / S_{t-1}) - 1
    spot_return = currency_list[spot_col].pct_change()

    # 3. Calculate the Funding Term (Cost of Carry)
    # Convert FW Pts (in BPS/Pips) to a decimal amount
    fwd_decimal_amount = currency_list[fwd_col] / divisor
    
    # We use Points_{t-1} because that was the price of carry agreed upon last month.
    # We divide by Spot_{t-1} to match the denominator of the spot_return.
    # Funding Term = (Fwd Decimal Amount) / Spot Price
    funding_term = fwd_decimal_amount.shift(1) / currency_list[spot_col].shift(1)
    
    # 4. Calculate the Excess Return
    # Note: If Points are Positive (Foreign Rate < US Rate), Funding Term is positive.
    # We SUBTRACT the funding cost (Paying the points).
    # If Points are Negative (Foreign Rate > US Rate), Funding Term is negative.
    # Subtracting a negative adds the yield (Earning the carry).
    # Excess Return = Spot Return - Funding Term
    excess_return = spot_return - funding_term
    
    # Rename and append
    excess_return.name = f'{country} Excess Return'
    excess_returns_list.append(excess_return)

# 5. Aggregate the Excess Returns DataFrame
excess_returns_df = pd.concat(excess_returns_list, axis=1)

# ADD THE US AS ZERO
# This ensures that if the US has the best macro data, 
# the model can allocate weight to "Cash" (USD).
excess_returns_df['US Excess Return'] = 0.0

# Drop the first row which contains NaN due to pct_change()
excess_returns_df.dropna(how='all', inplace=True)

# Display the resulting DataFrame
display(excess_returns_df)

,AU Excess Return,JP Excess Return,CH Excess Return,EU Excess Return,UK Excess Return,US Excess Return
Date,,,,,,
1971-01-31,NaN,NaN,NaN,NaN,NaN,0.0
1971-02-28,NaN,NaN,NaN,NaN,NaN,0.0
1971-03-31,NaN,NaN,NaN,NaN,NaN,0.0
1971-04-30,NaN,NaN,NaN,NaN,NaN,0.0
1971-05-31,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
2025-06-30,0.022894,70.301023,0.039504,0.036845,0.020143,0.0
2025-07-31,-0.024334,76.377791,-0.021028,-0.033740,-0.038378,0.0
2025-08-31,0.017326,78.213144,0.017267,0.021680,0.022185,0.0


## Portfolio Construction

In [15]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

# --- Equities ---
ranks_equities = business_cyle_signal_equities.rank(axis = 1, method = 'average')
display(ranks_equities)

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0,2.0,6.0,5.0,4.0,3.0
2025-07-31,2.0,1.0,6.0,5.0,4.0,3.0
2025-08-31,2.0,1.0,6.0,5.0,4.0,3.0


In [16]:
# --- Currencies ---
ranks_currencies = business_cyle_signal_currencies.rank(axis = 1, method = 'average')
display(ranks_currencies)

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0,3.0,5.0,6.0,2.0,4.0
2025-07-31,1.0,5.0,4.0,6.0,2.0,3.0
2025-08-31,1.0,5.0,4.0,6.0,2.0,3.0


In [17]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

# --- Equities ---
rank_means_equities = ranks_equities.mean(axis = 1)
rank_stds_equities = ranks_equities.std(axis = 1)
standardized_weights_equities = ranks_equities.sub(rank_means_equities, axis = 0).div(rank_stds_equities, axis = 0)

display(standardized_weights_equities)

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.336306,-0.801784,1.336306,0.801784,0.267261,-0.267261
2025-07-31,-0.801784,-1.336306,1.336306,0.801784,0.267261,-0.267261
2025-08-31,-0.801784,-1.336306,1.336306,0.801784,0.267261,-0.267261


In [18]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret_equities = (standardized_weights_equities.shift(1) * equity_returns).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol_equities = strategy_raw_ret_equities.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor_equities = target_vol / expected_vol_equities.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights_equities = standardized_weights_equities.mul(lev_factor_equities, axis=0)

display(final_strategy_weights_equities.dropna())

# # Step D: Final Strategy Returns
# # Compute the return by multiplying the final scaled weights (shifted) 
# # by the country returns.
# # The result is the aggregated portfolio return Series.
# final_strategy_ret = (final_strategy_weights.shift(1) * equity_returns).sum(axis=1)

# display("Final Strategy Returns (Aggregated):")
# display(final_strategy_ret)

,US,UK,CH,JP,AU,EU
1998-04-30,-0.381066,0.127022,0.635110,-0.635110,0.381066,-0.127022
1998-05-31,-0.126857,-0.380572,0.634287,-0.634287,0.380572,0.126857
1998-06-30,-0.125532,-0.376596,0.627660,-0.627660,0.376596,0.125532
1998-07-31,-0.124648,-0.373944,0.623240,-0.623240,0.373944,0.124648
1998-08-31,0.120463,-0.361390,0.602317,-0.120463,-0.602317,0.361390
...,...,...,...,...,...,...
2025-06-30,-0.601528,-0.360917,0.601528,0.360917,0.120306,-0.120306
2025-07-31,-0.358943,-0.598239,0.598239,0.358943,0.119648,-0.119648
2025-08-31,-0.360041,-0.600068,0.600068,0.360041,0.120014,-0.120014
2025-09-30,-0.361630,-0.602716,0.602716,0.361630,0.120543,-0.120543


In [19]:
# --- Currencies ---
rank_means_currencies = ranks_currencies.mean(axis = 1)
rank_stds_currencies = ranks_currencies.std(axis = 1)
standardized_weights_currencies = ranks_currencies.sub(rank_means_currencies, axis = 0).div(rank_stds_currencies, axis = 0)

display(standardized_weights_currencies)

,US,UK,CH,JP,AU,EU
observation_date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.336306,-0.267261,0.801784,1.336306,-0.801784,0.267261
2025-07-31,-1.336306,0.801784,0.267261,1.336306,-0.801784,-0.267261
2025-08-31,-1.336306,0.801784,0.267261,1.336306,-0.801784,-0.267261


In [20]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * equity_returns).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# Step C: Calculate Leverage Factor
target_vol = 0.10  # 10% Volatility Target

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0)

display("Final Strategy Weights (Scaled):")
display(final_strategy_weights)

# Step D: Final Strategy Returns
# Compute the return by multiplying the final scaled weights (shifted) 
# by the country returns.
# The result is the aggregated portfolio return Series.
final_strategy_ret = (final_strategy_weights.shift(1) * equity_returns).sum(axis=1)

display("Final Strategy Returns (Aggregated):")
display(final_strategy_ret)

NameError: name 'standardized_weights' is not defined

In [ ]:
equity_returns=equity_returns.loc["1998-04-30":"2025-10-31"]

In [ ]:
results_equities = pmp.run_cc_strategy(
    weights = final_strategy_weights_equities,
    returns = equity_returns,
    rf = rf,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

/Users/ryanyo/Developer/Backtesting for Portfolio Management/PMP/Macro Momentum/pmp_functions_v4.py:443: RuntimeWarning: invalid value encountered in scalar multiply
  tcost = turnover * t_cost
/Users/ryanyo/Developer/Backtesting for Portfolio Management/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ryanyo/Developer/Backtesting for Portfolio Management/PMP/Macro Momentum/pmp_functions_v4.py:443: RuntimeWarning: invalid value encountered in scalar multiply
  tcost = turnover * t_cost
/Users/ryanyo/Developer/Backtesting for Portfolio Management/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/ryanyo/Developer/Backtesting for Portfolio Management/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeW

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_UK,w_CH,w_JP,w_AU,w_EU
Date,,,,,,,,,,,,
1970-02-28,0.000000,0.000000,0.0,0.000000,0.0,"RF 0.0062 Name: 1970-02-28 00:00:00, dtype:...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-03-31,0.000000,0.000000,0.0,0.000000,0.0,"RF 0.0057 Name: 1970-03-31 00:00:00, dtype:...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-04-30,0.000000,0.000000,0.0,0.000000,0.0,"RF 0.005 Name: 1970-04-30 00:00:00, dtype: ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-05-31,0.000000,0.000000,0.0,0.000000,0.0,"RF 0.0053 Name: 1970-05-31 00:00:00, dtype:...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1970-06-30,0.000000,0.000000,0.0,0.000000,0.0,"RF 0.0058 Name: 1970-06-30 00:00:00, dtype:...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-31,-0.036715,-0.036715,0.0,0.082751,0.0,"RF 0.0034 Name: 2025-07-31 00:00:00, dtype:...",-0.564753,-0.331418,0.548271,0.334816,0.116913,-0.103829
2025-08-31,0.023740,0.023740,0.0,0.282640,0.0,"RF 0.0038 Name: 2025-08-31 00:00:00, dtype:...",-0.330012,-0.556737,0.553621,0.338966,0.107413,-0.113251
2025-09-30,-0.016850,-0.016850,0.0,0.080122,0.0,"RF 0.0033 Name: 2025-09-30 00:00:00, dtype:...",-0.337776,-0.551669,0.548656,0.339579,0.111765,-0.110555


## Performance Statistics

In [ ]:
pmp.run_perf_summary_benchmark_vs_strategy(results_equities, alreadyXs= True)

ValueError: Length of values (2) does not match length of index (1)

In [ ]:
factors_data = factors_data[["Mkt-RF", "SMB", "HML"]]
pmp.run_factor_regression(results_equities, factors_data, alreadyXs=True)

MissingDataError: exog contains inf or nans